In [3]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import time

ol = Overlay("sha1_5/sha1_5.bit")
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel
hls_ip = ol.axi_dma

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x01)

example_lst = [ord("h"), ord("e"), ord("l"), ord("l"), ord("o")]
hw_stream_output_lst = []
data_size = 100
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
for i in range(0,5):
    input_buffer[i] = example_lst[i]
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
current_time_hw_stream = time.time()
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
end_time_hw_stream = time.time()
hw_stream_output = ""
for i in range(5):
    hw_stream_output_lst.append(format(output_buffer[i], '02x'))
    hw_stream_output += format(output_buffer[i], '02x')
    
del input_buffer, output_buffer
print(hw_stream_output)
print("Expected output: aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d")
print("Hardware stream time taken with input size of 5:", end_time_hw_stream - current_time_hw_stream, "seconds")

aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
Expected output: aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
Hardware stream time taken with input size of 5: 0.001847982406616211 seconds


In [5]:
import struct
import hashlib
import time

def sha1(input_message):
   """
   Returns a hash produced based on the SHA-1
   :param input_message: string input
   :return: hexadecimal string of length 40 of 32 hexadecimal digits with total length of 160 bits
   """

   def left_rotate(n, b):
       return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

   h0 = 0x67452301
   h1 = 0xEFCDAB89
   h2 = 0x98BADCFE
   h3 = 0x10325476
   h4 = 0xC3D2E1F0

   encoded_message = input_message.encode('UTF-8')
   message_length = len(encoded_message)
   encoded_message += b'\x80'
   while len(encoded_message) % 64 != 56:
       encoded_message += b'\x00'
   encoded_message += struct.pack('>Q', message_length * 8)

   # Break the message up into 512-bit chunks
   for i in range(0, len(encoded_message), 64):
       chunk = encoded_message[i:i+64]
       w = [0] * 80
       # Break each chunk into 16 32-bit big-endian words
       for j in range(16):
           w[j] = struct.unpack('>I', chunk[j*4:j*4+4])[0]
       # Extend the 16 32-bit words into 80 32-bit words
       for j in range(16, 80):
           w[j] = left_rotate(w[j-3] ^ w[j-8] ^ w[j-14] ^ w[j-16], 1)

       # Initialize hash values for this chunk
       a = h0
       b = h1
       c = h2
       d = h3
       e = h4

       for j in range(80):
           if j < 20:
               f = (b & c) | ((~b) & d)
               k = 0x5A827999
           elif j < 40:
               f = b ^ c ^ d
               k = 0x6ED9EBA1
           elif j < 60:
               f = (b & c) | (b & d) | (c & d)
               k = 0x8F1BBCDC
           else:
               f = b ^ c ^ d
               k = 0xCA62C1D6

           temp = left_rotate(a, 5) + f + e + k + w[j] & 0xFFFFFFFF
           e = d
           d = c
           c = left_rotate(b, 30)
           b = a
           a = temp

       # Add current chunk's hash to the result
       h0 = (h0 + a) & 0xFFFFFFFF
       h1 = (h1 + b) & 0xFFFFFFFF
       h2 = (h2 + c) & 0xFFFFFFFF
       h3 = (h3 + d) & 0xFFFFFFFF
       h4 = (h4 + e) & 0xFFFFFFFF

   # return hash value
   return '%08x%08x%08x%08x%08x' % (h0, h1, h2, h3, h4)

# test output
input_string = 'hello'
current_time = time.time()
print("Software output:", sha1(input_string))
end_time = time.time()
print("Software time taken with input size of 5:", end_time - current_time, "seconds")

Software output: aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
Software time taken with input size of 5: 0.0032939910888671875 seconds


In [37]:
ol = Overlay("sha1_10/sha1_10.bit")
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel
hls_ip = ol.axi_dma

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x01)

example_lst = [ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o")]
hw_stream_output_lst = []
data_size = 100
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
for i in range(0,10):
    input_buffer[i] = example_lst[i]
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
current_time_hw_stream = time.time()
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
end_time_hw_stream = time.time()
hw_stream_output = ""
for i in range(5):
    hw_stream_output_lst.append(format(output_buffer[i], '02x'))
    hw_stream_output += format(output_buffer[i], '02x')
    
del input_buffer, output_buffer
print(hw_stream_output)
print("Expected output: b156215b189103c3d268f61299a854cd0b31e70")
print("Hardware stream time taken with input size of 10:", end_time_hw_stream - current_time_hw_stream, "seconds")

b156215b189103c3d268f61299a854cd0b31e70
Expected output: b156215b189103c3d268f61299a854cd0b31e70
Hardware stream time taken with input size of 10: 0.0018630027770996094 seconds


In [10]:
input_string = 'hellohello'
current_time = time.time()
print("Software output:", sha1(input_string))
end_time = time.time()
print("Software time taken with input size of 10:", end_time - current_time, "seconds")

Software output: 0b156215b189103c3d268f61299a854cd0b31e70
Software time taken with input size of 10: 0.0036308765411376953 seconds


In [19]:
ol = Overlay("sha1_20/sha1_20.bit")
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel
hls_ip = ol.axi_dma

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x01)

example_lst = [ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o")]
hw_stream_output_lst = []
data_size = 100
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
for i in range(0,20):
    input_buffer[i] = example_lst[i]
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
current_time_hw_stream = time.time()
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
end_time_hw_stream = time.time()
hw_stream_output = ""
for i in range(5):
    hw_stream_output_lst.append(format(output_buffer[i], '02x'))
    hw_stream_output += format(output_buffer[i], '02x')
    
del input_buffer, output_buffer
print(hw_stream_output)
print("Expected output: 8f3a9cbd5cb154b72b831afc87b4d4965e78afe3")
print("Hardware stream time taken with input size of 20:", end_time_hw_stream - current_time_hw_stream, "seconds")

8f3a9cbd5cb154b72b831afc87b4d4965e78afe3
Expected output: 8f3a9cbd5cb154b72b831afc87b4d4965e78afe3
Hardware stream time taken with input size of 20: 0.0019466876983642578 seconds


In [16]:
input_string = 'hellohellohellohello'
current_time = time.time()
print("Software output:", sha1(input_string))
end_time = time.time()
print("Software time taken with input size of 20:", end_time - current_time, "seconds")

Software output: 8f3a9cbd5cb154b72b831afc87b4d4965e78afe3
Software time taken with input size of 20: 0.005347251892089844 seconds


In [24]:
ol = Overlay("sha1_50/sha1_50.bit")
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel
hls_ip = ol.axi_dma

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x01)

example_lst = [ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o")]
hw_stream_output_lst = []
data_size = 200
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
for i in range(0,50):
    input_buffer[i] = example_lst[i]
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
current_time_hw_stream = time.time()
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
end_time_hw_stream = time.time()
hw_stream_output = ""
for i in range(5):
    hw_stream_output_lst.append(format(output_buffer[i], '02x'))
    hw_stream_output += format(output_buffer[i], '02x')
    
del input_buffer, output_buffer
print(hw_stream_output)
print("Expected output: 7b617da32c5bf8ef0766fed53c544fbbdf1c8b16")
print("Hardware stream time taken with input size of 50:", end_time_hw_stream - current_time_hw_stream, "seconds")

7b617da32c5bf8ef766fed53c544fbbdf1c8b16
Expected output: 7b617da32c5bf8ef0766fed53c544fbbdf1c8b16
Hardware stream time taken with input size of 50: 0.0019524097442626953 seconds


In [21]:
input_string = 'hellohellohellohellohellohellohellohellohellohello'
current_time = time.time()
print("Software output:", sha1(input_string))
end_time = time.time()
print("Software time taken with input size of 50:", end_time - current_time, "seconds")

Software output: 7b617da32c5bf8ef0766fed53c544fbbdf1c8b16
Software time taken with input size of 50: 0.005620718002319336 seconds


In [29]:
ol = Overlay("sha1_100/sha1_100.bit")
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel
hls_ip = ol.axi_dma

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x01)

example_lst = [ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o"),
               ord("h"), ord("e"), ord("l"), ord("l"), ord("o"), ord("h"), ord("e"), ord("l"), ord("l"), ord("o")]
hw_stream_output_lst = []
data_size = 400
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
for i in range(0,100):
    input_buffer[i] = example_lst[i]
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
current_time_hw_stream = time.time()
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
end_time_hw_stream = time.time()
hw_stream_output = ""
for i in range(5):
    hw_stream_output_lst.append(format(output_buffer[i], '02x'))
    hw_stream_output += format(output_buffer[i], '02x')
    
del input_buffer, output_buffer
print(hw_stream_output)
print("Expected output: afcefbe8bac7a70a11cc9252cc8f322aac715328")
print("Hardware stream time taken with input size of 100:", end_time_hw_stream - current_time_hw_stream, "seconds")

afcefbe8bac7a70a11cc9252cc8f322aac715328
Expected output: afcefbe8bac7a70a11cc9252cc8f322aac715328
Hardware stream time taken with input size of 100: 0.001955747604370117 seconds


In [29]:
input_string = 'hellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohellohello'
current_time = time.time()
print("Software output:", sha1(input_string))
end_time = time.time()
print("Software time taken with input size of 100:", end_time - current_time, "seconds")

Software output: afcefbe8bac7a70a11cc9252cc8f322aac715328
Software time taken with input size of 100: 0.0058672428131103516 seconds
